In [ ]:
from kan import KAN

import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torch

import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import *

# <center>Pré-processamento dos dados</center>

In [ ]:
train_path = 'Dataset/train'
test_path = 'Dataset/test'

In [ ]:
#Aumentar a quantidade de técnicas de aumento de dados
transform_train = transforms.Compose(
    [transforms.Resize(size = (164,164)),
     transforms.RandomRotation(degrees=15),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5], std=[0.5])
    ]
)

transform_test = transforms.Compose(
    [transforms.Resize(size = (164,164)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5], std=[0.5])
    ]
)

In [ ]:
train_dataset = datasets.ImageFolder(train_path, transform = transform_train)
test_dataset = datasets.ImageFolder(test_path, transform = transform_test)

In [ ]:
trainloader = DataLoader(train_dataset, shuffle = True)
testloader = DataLoader(test_dataset, shuffle = True)

In [ ]:
train_data = []
train_targets = []

for batch_data_train, batch_targets_train in trainloader:
    train_data.append(batch_data_train)
    train_targets.append(batch_targets_train)

train_data = torch.cat(train_data, dim=0)
train_targets = torch.cat(train_targets, dim=0)

In [ ]:
print("Targets train:")
print(train_targets)

In [ ]:
test_data = []
test_targets = []

for batch_data_test, batch_targets_test in testloader:
    test_data.append(batch_data_test)
    test_targets.append(batch_targets_test)

test_data = torch.cat(test_data, dim=0)
test_targets = torch.cat(test_targets, dim=0)

In [ ]:
print("Targets test:")
print(test_targets)

In [ ]:
dataset = {}

dataset['train_input'] = train_data.view(-1, 164*164*3)
dataset['test_input'] = test_data.view(-1, 164*164*3)
dataset['train_label'] = train_targets
dataset['test_label'] = test_targets

# <center>Implementação da arquitetura KAN</center>

In [ ]:
model = KAN(width=[dataset['train_input'].shape[1], 14, 3], grid=3, k=3, base_fun=torch.nn.ReLU(), auto_save=False)

In [ ]:
dtype = torch.get_default_dtype()

def train_acc():
    return torch.mean((torch.argmax(model(dataset['train_input']), dim=1) == dataset['train_label']).type(dtype))

def test_acc():
    return torch.mean((torch.argmax(model(dataset['test_input']), dim=1) == dataset['test_label']).type(dtype))


In [ ]:
results = model.fit(dataset, opt="Adam", steps=250, metrics=(train_acc, test_acc), 
                    loss_fn=torch.nn.CrossEntropyLoss(), lr=0.001, batch=30)

In [ ]:
results['train_acc'][-1], results['test_acc'][-1]

In [ ]:
epochs = range(1, 251)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, results['train_loss'], 'bo-')
plt.xlabel('Épocas')
plt.ylabel('Perda')

plt.subplot(1, 2, 2)
plt.plot(epochs, results['train_acc'], 'ro-')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')

plt.suptitle("Treinamento", fontsize = 20)

In [ ]:
epochs = range(1, 251)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs, results['test_loss'], 'bo-')
plt.xlabel('Épocas')
plt.ylabel('Perda')

plt.subplot(1, 2, 2)
plt.plot(epochs, results['test_acc'], 'ro-')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')

plt.suptitle("Teste", fontsize = 20)

In [ ]:
test_outputs = model(dataset['test_input'])
test_predictions = torch.argmax(test_outputs, dim=1).numpy()

# <center>Métricas de avaliação</center>

In [ ]:
cm = confusion_matrix(dataset['test_label'], test_predictions)

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Normal', 'Pneumonia', 'Tuberculose'])
disp.plot(cmap=plt.cm.Blues)

plt.xlabel('Rótulo previsto')
plt.ylabel('Rótulo verdadeiro')
plt.show()

In [ ]:
accuracy = accuracy_score(dataset['test_label'], test_predictions)
print(f"Acurácia: {accuracy:.2f}")

In [ ]:
precision = precision_score(dataset['test_label'], test_predictions, average='weighted')
print(f"Precisão: {precision:.2f}")

In [ ]:
recall = recall_score(dataset['test_label'], test_predictions, average='weighted')
print(f"Recall: {recall:.2f}")

In [ ]:
f1 = f1_score(dataset['test_label'], test_predictions, average='weighted')
print(f"F1 Score: {f1:.2f}")